In [4]:
import pandas as pd
from utils import get_engine
from dotenv import load_dotenv
from sqlalchemy import text
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from utils import get_df_from_db

In [5]:
load_dotenv()
engine = get_engine(echo_arg=True)

In [6]:
df = get_df_from_db(engine)

2023-06-20 14:29:37,514 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-06-20 14:29:37,516 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-20 14:29:37,559 INFO sqlalchemy.engine.Engine select current_schema()
2023-06-20 14:29:37,562 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-20 14:29:37,608 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-06-20 14:29:37,611 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-06-20 14:29:37,651 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-20 14:29:37,663 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

2023-06-20 14:29:37,925 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from interpret.glassbox import LogisticRegression
from interpret import show

In [8]:
y1, y2 = df['TotalGHGEmissions_log'], df['SiteEnergyUse_kBtu_log']
X_cols = ['Have_Stream_Energy','Have_NaturalGas_Energy', 'PrimaryPropertyType', 
          'NumberofBuildings','LargestPropertyUseTypeGFA_log']
X = df[X_cols]
bool_cols = ['Have_Stream_Energy', 'Have_NaturalGas_Energy']
X[bool_cols] = X[bool_cols].apply(lambda x: x.apply(lambda x: 1 if x==True else 0))

/tmp/ipykernel_541175/582719829.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[bool_cols] = X[bool_cols].apply(lambda x: x.apply(lambda x: 1 if x==True else 0))


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from interpret.glassbox import ExplainableBoostingRegressor, RegressionTree
ebm = ExplainableBoostingRegressor()
ebm2 = RegressionTree()
X_cat = X.select_dtypes(include=[object])
X_num = X.select_dtypes(exclude=[object])
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.2, random_state=42)
preparation = ColumnTransformer(transformers=[
    ('tf_cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), X_cat.columns),
    ('tf_num', RobustScaler(), X_num.columns)
])

In [10]:
from interpret import show
X_tf = preparation.fit_transform(X_train)
names = preparation.get_feature_names_out(['Have_Stream_Energy','Have_NaturalGas_Energy', 'PrimaryPropertyType', 
          'NumberofBuildings','LargestPropertyUseTypeGFA_log'])
print(names)
df_process = pd.DataFrame(data=X_tf, columns=names)
ebm2.fit(df_process, y_train)
ebm_global = ebm2.explain_global()
show(ebm_global)

['tf_cat__PrimaryPropertyType_Distribution Center'
 'tf_cat__PrimaryPropertyType_High-Rise Multifamily'
 'tf_cat__PrimaryPropertyType_Hospital'
 'tf_cat__PrimaryPropertyType_Hotel'
 'tf_cat__PrimaryPropertyType_K-12 School'
 'tf_cat__PrimaryPropertyType_Laboratory'
 'tf_cat__PrimaryPropertyType_Large Office'
 'tf_cat__PrimaryPropertyType_Low-Rise Multifamily'
 'tf_cat__PrimaryPropertyType_Medical Office'
 'tf_cat__PrimaryPropertyType_Mid-Rise Multifamily'
 'tf_cat__PrimaryPropertyType_Mixed Use Property'
 'tf_cat__PrimaryPropertyType_Office' 'tf_cat__PrimaryPropertyType_Other'
 'tf_cat__PrimaryPropertyType_Refrigerated Warehouse'
 'tf_cat__PrimaryPropertyType_Residence Hall'
 'tf_cat__PrimaryPropertyType_Restaurant'
 'tf_cat__PrimaryPropertyType_Retail Store'
 'tf_cat__PrimaryPropertyType_Self-Storage Facility'
 'tf_cat__PrimaryPropertyType_Senior Care Community'
 'tf_cat__PrimaryPropertyType_Small- and Mid-Sized Office'
 'tf_cat__PrimaryPropertyType_Supermarket / Grocery Store'
 'tf_c

<!-- http://127.0.0.1:7001/140683006760512/ -->